In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize, FreqDist, BigramCollocationFinder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string


### Objective:
Guide is: [article](https://towardsdatascience.com/sentiment-analysis-and-product-recommendation-on-amazons-electronics-dataset-reviews-part-1-6b340de660c2)
Set up a 'dev doc' where we can 

The code below comes from where I got the data. It was a helper function provided to help with ease of access.

In [2]:
import pandas as pd
import gzip
import json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
df = getDF('Magazine_Subscriptions.json.gz')

In [4]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89689 entries, 0 to 89688
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         89689 non-null  float64
 1   vote            24103 non-null  object 
 2   verified        89689 non-null  bool   
 3   reviewTime      89689 non-null  object 
 4   reviewerID      89689 non-null  object 
 5   asin            89689 non-null  object 
 6   reviewerName    89687 non-null  object 
 7   reviewText      89656 non-null  object 
 8   summary         89670 non-null  object 
 9   unixReviewTime  89689 non-null  int64  
 10  style           51398 non-null  object 
 11  image           135 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 8.3+ MB


In [6]:
to_drop = ['image', 'style']
df.vote.fillna(0, inplace=True)
df.reviewerName.fillna('Amazon Customer', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89689 entries, 0 to 89688
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         89689 non-null  float64
 1   vote            89689 non-null  object 
 2   verified        89689 non-null  bool   
 3   reviewTime      89689 non-null  object 
 4   reviewerID      89689 non-null  object 
 5   asin            89689 non-null  object 
 6   reviewerName    89687 non-null  object 
 7   reviewText      89656 non-null  object 
 8   summary         89670 non-null  object 
 9   unixReviewTime  89689 non-null  int64  
 10  style           51398 non-null  object 
 11  image           135 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 8.3+ MB


In [19]:
results = {}
for column in df.columns:
    results[column] = df[column].value_counts()
results

{'overall': 5.0    53790
 4.0    12676
 1.0    11029
 3.0     6971
 2.0     5223
 Name: overall, dtype: int64,
 'vote': 0      65586
 2       6318
 3       3657
 4       2403
 5       1754
        ...  
 420        1
 469        1
 284        1
 228        1
 257        1
 Name: vote, Length: 288, dtype: int64,
 'verified': True     58654
 False    31035
 Name: verified, dtype: int64,
 'reviewTime': 02 20, 2015    146
 03 29, 2016    131
 03 25, 2014    127
 05 30, 2015    114
 03 1, 2016     113
               ... 
 03 10, 2004      1
 01 31, 2007      1
 12 27, 2005      1
 06 12, 2005      1
 07 2, 2006       1
 Name: reviewTime, Length: 5804, dtype: int64,
 'reviewerID': A3JPFWKS83R49V    55
 A2OTUWUSH49XIN    26
 AEMZRE6QYVQBS     25
 A3GA09FYFKL4EY    24
 A3R7MXVQRGGIQ9    22
                   ..
 A1DRKHXDHIJUKL     1
 A11DO937IE4S6T     1
 A1RBMSUX4LG0O1     1
 AC3MTD3XECI64      1
 A1EPYR0EF39PMB     1
 Name: reviewerID, Length: 72098, dtype: int64,
 'asin': B00005NIOH    1718

In [9]:
stopwords_list = stopwords.words('english') + list(string.punctuation) + ["''", '""', '...', '``', "'s", "n't"]
def process_article(article):
    return [w.lower() for w in word_tokenize(article) if w.lower() not in stopwords_list]

In [10]:
data = list(df.reviewText.dropna())

In [11]:
processed_data = list(map(process_article, data))


In [12]:
out = False
if 'not' in stopwords_list:
    out = True
out

True

In [13]:
total_vocab = set()
for article in processed_data:
    total_vocab.update(article)
len(total_vocab)

60908

In [14]:
review_concat = []
for review in processed_data:
    review_concat += review
review_freqdist = FreqDist(review_concat)
review_freqdist.most_common(200)
total_word_count = sum(review_freqdist.values())

In [15]:
normalized_freqs = {}
for word in review_freqdist.most_common(200):
    normalized_freq = word[1]/total_word_count
    normalized_freqs[word[0]] = normalized_freq


In [16]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(review_concat)
scored = finder.score_ngrams(bigram_measures.raw_freq)
scored[:50]

[(('great', 'magazine'), 0.0023747449472953544),
 (('love', 'magazine'), 0.0023017835074163303),
 (('first', 'issue'), 0.0012073619092309714),
 (('magazine', 'great'), 0.0011598869997206476),
 (('good', 'magazine'), 0.0008745378067691225),
 (('look', 'forward'), 0.0008685409760941342),
 (('every', 'month'), 0.0007855848184234632),
 (('every', 'issue'), 0.0007830861389755514),
 (('like', 'magazine'), 0.0006751431868257626),
 (('magazine', 'love'), 0.0006081785776217269),
 (('cover', 'cover'), 0.0006056798981738152),
 (('years', 'ago'), 0.0005921870291550915),
 (('kindle', 'fire'), 0.000578694160136368),
 (('great', 'articles'), 0.0005472107990926795),
 (('year', 'old'), 0.0005432129119760207),
 (('well', 'written'), 0.0005332181941843736),
 (('great', 'price'), 0.0005197253251656499),
 (('magazine', 'good'), 0.0005192255892760676),
 (('interesting', 'articles'), 0.0005012350972511028),
 (('highly', 'recommend'), 0.0004897411717907086),
 (('many', 'years'), 0.0004852435487844674),
 (('re

In [17]:
pmi_finder = BigramCollocationFinder.from_words(review_concat)
pmi_finder.apply_freq_filter(5)
review_pmi_scored = pmi_finder.score_ngrams(bigram_measures.pmi)
review_pmi_scored[:50]

[(('dame', 'edna'), 18.34736833152189),
 (('settimana', 'enigmistica'), 18.34736833152189),
 (('humpty', 'dumpty'), 18.124975910185444),
 (('bells', 'whistles'), 18.12497591018544),
 (('miley', 'cyrus'), 18.12497591018544),
 (('fons', 'porter'), 17.93233083224305),
 (('gwyneth', 'paltrow'), 17.93233083224305),
 (('a-section', 'a-spacing-small'), 17.932330832243046),
 (('a-spacing-small', 'a-spacing-top-mini'), 17.932330832243046),
 (('a-spacing-top-mini', 'video-block'), 17.932330832243046),
 (('roz', 'chast'), 17.932330832243046),
 (('video-block', '/div'), 17.932330832243046),
 (('bona', 'fide'), 17.902583488848993),
 (('christina', 'aguilera'), 17.861941504351652),
 (('sofia', 'vergara'), 17.861941504351652),
 (('movers', 'shakers'), 17.762405830800734),
 (('suze', 'orman'), 17.76240583080073),
 (('puerto', 'rico'), 17.739685754300652),
 (('di', 'giacomo'), 17.610402737355688),
 (('div', 'id='), 17.610402737355685),
 (('fareed', 'zakaria'), 17.610402737355685),
 (('tai', 'chi'), 17.